In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
stu_ques = pd.read_csv("../dataset/person_and_question_relationship.csv",header=0)
question_information = pd.read_csv("../dataset/question_information.csv",header=0)

In [2]:
stu_que_matrix = pd.merge(stu_ques, question_information, how='left')
stu_que_matrix.columns.values

array(['itest_id', 'problemid', 'actioncount', 'bottomhint', 'correct',
       'correctrate', 'dayfinishquestion', 'end_time_day',
       'end_time_month', 'endtime', 'filedin', 'frhrate', 'frscaffoldh',
       'hint', 'hintrate', 'monthfinishquestion', 'scaffold',
       'timefinishquestion', 'timeover80', 'timeover80rate', 'timetaken',
       'timetohint', 'workinschoolrate', 'wrong', 'wrongrate', 'skill'],
      dtype=object)

把学生和skill联系起来，每个skill正确率大于0.5的学生多少人，小于0.5的学生多少人

正确率大于0.5且去了stem的学生多少人，小于0.5且去了stem的学生多少人

正确率大于0.5且没去stem的学生多少人，小于0.5且没去stem的学生多少人

我们可以评估skill的难度

In [3]:
stu_que_matrix = pd.merge(stu_ques, question_information, how='left')
from tqdm import tqdm
train=pd.read_csv('training_label.csv',header=0)
train = train.rename(columns={'ITEST_id':'itest_id'})
train=train.set_index('itest_id')
# print(train)
# stu_que_matrix = pd.merge(stu_ques, question_information, how='left')
# stu_que_matrix=stu_que_matrix.set_index('itest_id')
# print(stu_que_matrix)
# sq=pd.merge(stu_que_matrix,train,how='left')
ls=np.ndarray.tolist(stu_que_matrix.skill.values)
ls3=[]
lls=[]
lls1=[]
lls2=[]
lls3=[]
lls4=[]
lls5=[]
ls2 = list(set(ls))
#print(ls2)
for i in tqdm(range(len(ls2))):    
    skill=stu_que_matrix[stu_que_matrix.skill==ls2[i]]
    skill_df=skill.groupby('itest_id').mean()
    #print(skill_df)
    ls1 =skill_df.index
    ls1=ls1.tolist()
    #print(ls1)
    lss1=train.index
    lss1=lss1.tolist()
    #print(len(lss1))
    isStem=train['isSTEM'].values
    isStem=isStem.tolist()
    #print(len(isStem))
    #print(ls1)
    shape=skill_df.shape
    sh=shape[0]
    ls3.append(sh)
    n=np.array(ls3)
    length=0
    length1=0
    l=0
    l1=0
    l2=0
    l3=0
    for m in ls1:
        if skill_df.loc[m,'correctrate']>0.5:
            length+=1
            #for n in range(sh):
            if m in lss1:
                index=lss1.index(m)
                if isStem[index]==1:
                    l+=1
                elif isStem[index]==0:
                    l1+=1
        else:
            length1+=1            
            #for n in range(sh):
            if m in lss1:
                index=lss1.index(m)
                if isStem[index]==1:
                    l2+=1
                elif isStem[index]==0:
                    l3+=1
    lls.append(length)
    lls1.append(length1)
    lls2.append(l)
    lls3.append(l1)
    lls4.append(l2)
    lls5.append(l3)
# print("correct rate > 0.5:")
# print(lls)
# print("correct rate < 0.5:")
# print(lls1)
# print("correct rate is stem > 0.5:")
# print(lls2)
# print("correct rate is not stem > 0.5:")
# print(lls3)
# print("correct rate is stem < 0.5:")
# print(lls4)
# print("correct rate is not stem < 0.5:")
# print(lls5)

100%|████████████████████████████████████████████████████████████████████████████████| 102/102 [00:08<00:00, 12.68it/s]


In [4]:
df3 = pd.DataFrame({'skill':np.array(ls2),
                    'total_number' : np.array(ls3,dtype='int32'),
                    'morethan0.5' : np.array(lls,dtype='int32'),
                    'lessthan0.5':np.array(lls1,dtype='int32'),
                    'morethan0.5stem':np.array(lls2,dtype='int32'),
                    'morethan0.5notstem':np.array(lls3,dtype='int32'),
                    'lessthan0.5stem':np.array(lls4,dtype='int32'),
                    'lessthan0.5notstem':np.array(lls5,dtype='int32')})
#df3.to_csv('skill_and_student.csv')
df3=df3.set_index('skill')
#由于去了stem的人比较少，大概没去的是去了的两倍，所以乘以二算比例
df3['lessthan0.5stem']=df3['lessthan0.5stem']*2
df3['morethan0.5stem']=df3['morethan0.5stem']*2
#df3['lessratio']=(df3['lessthan0.5stem']-df3['lessthan0.5notstem'])/df3['lessthan0.5notstem']
#df3['moreratio']=(df3['morethan0.5stem']-df3['morethan0.5notstem'])/df3['morethan0.5notstem']
#df3['less-more']=(df3['lessthan0.5']-df3['morethan0.5'])/df3['total_number']
df3['less/total']=df3['lessthan0.5']/df3['total_number']
df3['train_nostem_rate']=df3['lessthan0.5notstem']/(df3['lessthan0.5notstem']+df3['morethan0.5notstem'])
df3['train_stem_rate']=df3['lessthan0.5stem']/(df3['lessthan0.5stem']+df3['morethan0.5stem'])
#df3['less0.5nostem/total']=df3['lessthan0.5']/df3['total_number']
#df3['more/total']=df3['morethan0.5']/df3['total_number']
df3

,lessthan0.5,lessthan0.5notstem,lessthan0.5stem,morethan0.5,morethan0.5notstem,morethan0.5stem,total_number,less/total,train_nostem_rate,train_stem_rate
skill,,,,,,,,,,
linear-area-volume-conversion,588,119,72,337,79,92,925,0.635676,0.601010,0.439024
graph interpretation,182,33,12,189,43,32,371,0.490566,0.434211,0.272727
integers,104,20,18,380,91,76,484,0.214876,0.180180,0.191489
application: multi-column subtraction,0,0,0,1,0,0,1,0.000000,NaN,NaN
fractions,478,103,72,368,84,70,846,0.565012,0.550802,0.507042
circumference,63,14,12,22,8,10,85,0.741176,0.636364,0.545455
properties-of-solids,57,8,6,103,21,28,160,0.356250,0.275862,0.176471
supplementary-angles,698,139,60,439,90,90,1137,0.613896,0.606987,0.400000
reduce-fraction,43,6,2,29,6,14,72,0.597222,0.500000,0.125000


技能正确率少于0.5证明很难

根据之前计算的比率 ‘less/total’是每个技能正确率少于0.5学生人数/总学生人数

'train_nostem_rate'每个技能正确率少于0.5但是没去stem/总的没去stem的人数

'train_stem_rate'每个技能正确率少于0.5但是去了stem/总的没去stem的人数

去掉Null值，根据这三个指标分成三个簇类

In [9]:
from sklearn.cluster import KMeans
import json
df3 = df3.loc[:,['less/total','train_nostem_rate','train_stem_rate']]
data = df3.dropna(
    axis=0,     # 0: 对行进行操作; 1: 对列进行操作
    how='any'   # 'any': 只要存在 NaN 就 drop 掉; 'all': 必须全部是 NaN 才 drop 
    ).values 
#print(data)
#假如我要构造一个聚类数为3的聚类器
estimator = KMeans(n_clusters=3)#构造聚类器
estimator.fit(data)#聚类
label_pred = estimator.labels_ #获取聚类标签
centroids = estimator.cluster_centers_ #获取聚类中心
inertia = estimator.inertia_ # 获取聚类准则的总和
# print(label_pred)
print(centroids)
# print(inertia)
list_index0=[]
list_index1=[]
list_index2=[]
index_skill=df3.index.tolist()
find0=0
index0=[i for i,v in enumerate(label_pred) if v==find0]
find1=1
index1=[i for i,v in enumerate(label_pred) if v==find1]
find2=2
index2=[i for i,v in enumerate(label_pred) if v==find2]
for i in index0:
    m=index_skill[i]
    list_index0.append(m)
for i in index1:
    m=index_skill[i]
    list_index1.append(m) 
for i in index2:
    m=index_skill[i]
    list_index2.append(m)

[[0.63636177 0.62382776 0.54169649]
 [0.43338837 0.38563246 0.27639605]
 [0.23107495 0.19854853 0.1411294 ]]


In [10]:
dic = {'hard':list_index0,'middle':list_index1,'easy':list_index2}

j = json.dumps(dic,ensure_ascii=False)
print(type(j))
with open('skill_cluster.json','w') as outfile:
    json.dump(j,outfile)
dic

<class 'str'>


{'easy': ['integers',
  'circumference',
  'percents',
  'exponents',
  'interpreting-linear-equations',
  'algebra symbolization',
  'area',
  'rounding',
  'mean',
  'transformations-rotations',
  'noskill',
  'congruence',
  'application: find slope in graph',
  'pythagorean-theorem',
  'least-common-multiple',
  'rate',
  'subtraction',
  'scientific-notation'],
 'hard': ['linear-area-volume-conversion',
  'application: multi-column subtraction',
  'fractions',
  'properties-of-solids',
  'finding-percents',
  'sum-of-interior-angles-more-than-3-sides',
  'stem-and-leaf-plot',
  'application: isosceles triangle',
  'equation-concept',
  'p-patterns-relations-algebra',
  'slope',
  'multiplying-positive-negative-numbers',
  'circle-graph',
  'evaluating-functions',
  'equivalent-fractions-decimals-percents',
  'addition',
  'application: compare points',
  'reading-graph',
  'perimeter',
  'application: finding percentage of a number',
  'equation-solving',
  'algebraic-manipulation

之前保存的三个簇类里面的技能 分别把三个簇类命名为：class1 class2 class3

把每个人回答的所有的class1簇类里面的技能的特征提取出来

把每个人回答的所有的class2簇类里面的技能的特征提取出来

。。。。

一共生成三个dataframe

然后分别groupby，再合到一起（大概每个要20分钟）

In [11]:
class1= dic['middle']
df_class1 = stu_que_matrix[stu_que_matrix['skill'].isin(class1)]
print(len(df_class1))
class2= dic['hard']
df_class2 = stu_que_matrix[stu_que_matrix['skill'].isin(class2)]
print(len(df_class2))
class3= dic['easy']
df_class3 = stu_que_matrix[stu_que_matrix['skill'].isin(class3)]
print(len(df_class3))

184825
81035
95418


### 统计特征
第一类：mean 

分成4种： 3个class，1个全集

In [12]:
mean_col_names = ['actioncount', 'bottomhint', 'correct',
       'correctrate', 'filedin', 'frhrate', 'frscaffoldh',
       'hint', 'hintrate', 'scaffold',
       'timeover80', 'timeover80rate', 'timetaken',
       'timetohint', 'workinschoolrate', 'wrong', 'wrongrate']
all_mean = stu_que_matrix.groupby('itest_id').mean()[mean_col_names]
c1_mean = df_class1.groupby('itest_id').mean()[mean_col_names]
c2_mean = df_class2.groupby('itest_id').mean()[mean_col_names]
c3_mean = df_class3.groupby('itest_id').mean()[mean_col_names]

In [13]:
# add up
student_info_matrix = pd.read_csv("../dataset/student_information.csv").set_index('itest_id')
student_info_matrix = student_info_matrix.join(all_mean,how='outer')
student_info_matrix = student_info_matrix.join(c1_mean.rename(columns = lambda x:x+'_M'),how='outer')
student_info_matrix = student_info_matrix.join(c2_mean.rename(columns = lambda x:x+'_H'),how='outer')
student_info_matrix = student_info_matrix.join(c3_mean.rename(columns = lambda x:x+'_E'),how='outer')

第二类：时间统计

对做题天数（月数），每天（月）平均做题个数和方差进行统计

In [14]:
def create_date_info_dataframe(dataframe):
    group_by_day = dataframe.groupby(['itest_id','dayfinishquestion'])
    group_by_day1 = group_by_day.count()
    average_questions_by_day  = group_by_day1.groupby(['itest_id']).mean()['problemid']
    std_questions_by_day = group_by_day1.groupby(['itest_id']).std()['problemid']
    counts_day_answer = group_by_day1.groupby(['itest_id']).count()['problemid']

    group_by_mon = dataframe.groupby(['itest_id','monthfinishquestion'])
    group_by_mon1 = group_by_mon.count()
    average_questions_by_mon = group_by_mon1.groupby(['itest_id']).mean()['problemid']
    std_questions_by_mon = group_by_mon1.groupby(['itest_id']).std()['problemid']
    counts_mon_answer = group_by_mon1.groupby(['itest_id']).count()['problemid']
    
    data = {'average_questions_by_day':average_questions_by_day,'std_questions_by_day':std_questions_by_day,
           'counts_day_answer':counts_day_answer,'average_questions_by_mon':average_questions_by_mon,
           'std_questions_by_mon':std_questions_by_mon,'counts_mon_answer':counts_mon_answer}
    new_frame = pd.DataFrame(data)
    
    return new_frame

In [15]:
# all
all_date = create_date_info_dataframe(stu_que_matrix)
c1_date = create_date_info_dataframe(df_class1)
c2_date = create_date_info_dataframe(df_class2)
c3_date = create_date_info_dataframe(df_class3)
# add up
student_info_matrix = student_info_matrix.join(all_date,how='outer')
student_info_matrix = student_info_matrix.join(c1_date.rename(columns = lambda x:x+'_M'),how='outer')
student_info_matrix = student_info_matrix.join(c2_date.rename(columns = lambda x:x+'_H'),how='outer')
student_info_matrix = student_info_matrix.join(c3_date.rename(columns = lambda x:x+'_E'),how='outer')

第三类，仅与题目相关

比如，题目超过80秒但是还是答错的

In [16]:
def over80_but_fair(dataframe):
    dataframe['timetaken_over80'] = (dataframe['timetaken']>80).astype(int)
    dataframe['wrong_over80'] = ((dataframe['timetaken_over80'] > 0) & (dataframe['correctrate'] == 0)).astype(int)
    group1 = dataframe.groupby('itest_id')
    wrong_and_over80 = group1.sum()['wrong_over80'] / group1.count()['problemid']
    return wrong_and_over80

In [17]:
student_info_matrix['wrong_and_over80'] = over80_but_fair(stu_que_matrix)
student_info_matrix['wrong_and_over80_M'] = over80_but_fair(df_class1)
student_info_matrix['wrong_and_over80_H'] = over80_but_fair(df_class2)
student_info_matrix['wrong_and_over80_E'] = over80_but_fair(df_class3)

C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
student_info_matrix.to_csv("student_information_106.csv")